In [97]:
pip install numpy -q

In [98]:
pip install pandas -q

In [99]:
pip install scikit-learn

In [100]:
from google.colab import files
uploaded = files.upload()

Saving seeds.csv to seeds (3).csv


In [101]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score

In [102]:
df = pd.read_csv('seeds.csv')
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [103]:
df["V8"].unique()

array([1, 2, 3])

In [104]:
labels = df["V8"]
labels

,V8
0,1
1,1
2,1
3,1
4,1
...,...
205,3
206,3
207,3
208,3


In [105]:
df.drop("V8", axis=1, inplace=True)
df.head()

,V1,V2,V3,V4,V5,V6,V7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


In [106]:
dados = df.to_numpy()
dados

array([[15.26  , 14.84  ,  0.871 , ...,  3.312 ,  2.221 ,  5.22  ],
       [14.88  , 14.57  ,  0.8811, ...,  3.333 ,  1.018 ,  4.956 ],
       [14.29  , 14.09  ,  0.905 , ...,  3.337 ,  2.699 ,  4.825 ],
       ...,
       [13.2   , 13.66  ,  0.8883, ...,  3.232 ,  8.315 ,  5.056 ],
       [11.84  , 13.21  ,  0.8521, ...,  2.836 ,  3.598 ,  5.044 ],
       [12.3   , 13.34  ,  0.8684, ...,  2.974 ,  5.637 ,  5.063 ]])

In [107]:
def inicializao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

In [108]:
def atualizacao_medoids(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m  # preparação dos graus de pertinência
    n_clusters = matriz_pertinencia.shape[1]  # número de clusters
    #print(f"n_clusters na atualizao_medoids = {n_clusters}")
    medoids = np.zeros((n_clusters, dados.shape[1]))  # inicializa os medoids

    for k in range(n_clusters):
        # Seleciona os pontos com base nas pertinências elevadas para o cluster k
        pertinencias_m = matriz_pertinencia_m[:, k]

        # Calcula a soma ponderada das distâncias de cada ponto aos demais pontos do cluster
        custos = np.sum(pertinencias_m[:, np.newaxis] * np.linalg.norm(dados - dados[:, np.newaxis], axis=2), axis=0)

        # Seleciona o ponto com o menor custo como medoid
        medoid_idx = np.argmin(custos)
        medoids[k] = dados[medoid_idx]

    return medoids

In [109]:
def atualizacao_matriz_pertinencia(dados, medoids, m):
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - medoids, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

In [110]:
def fcmdd(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        medoids = atualizacao_medoids(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, medoids, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return medoids, matriz_pertinencia

In [111]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

In [112]:
def experimento_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        #print(_)
        medoids, matriz_pertinencia = fcmdd(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        #print(predicted_labels)
        idx_rand = indice_rand(labels, predicted_labels)
        indices_rand.append(idx_rand)
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)
    return mean_rand_index, std_rand_index

In [113]:
num_clusters = 3
num_trials = 100
media_indice_rand, dp_indice_rand = experimento_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}")
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}")

Monte Carlo FCM Clustering Results (100 trials)
Mean Rand Index: 0.5045
Standard Deviation of Rand Index: 0.1558
